In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error,explained_variance_score
import seaborn as sns

In [3]:
call_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/Call_with_svol_vix.csv')
put_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/Put_with_svol_vix.csv')
call_df=call_df.set_index(['Date'])
put_df=put_df.set_index(['Date'])

In [4]:
call_df.head()

,Strike Price,Close,Expiration,Adj Close,Price,bs_call,svol,Vix
Date,,,,,,,,
2013-12-31,6200,199.65,30,6304.0,8.82,188.621115,0.10855,15.08
2013-12-31,6300,131.00,30,6304.0,8.82,124.059028,0.10855,15.08
2013-12-31,6400,79.00,30,6304.0,8.82,75.195204,0.10855,15.08
2013-12-31,6500,41.35,30,6304.0,8.82,41.683668,0.10855,15.08
2013-12-31,6600,18.60,30,6304.0,8.82,21.015752,0.10855,15.08


In [5]:
put_df.head()

,Strike Price,Close,Expiration,Adj Close,Price,bs_call,svol,Vix
Date,,,,,,,,
2013-12-31,5800,6.30,30,6304.0,8.82,0.780877,0.10855,15.08
2013-12-31,5900,10.30,30,6304.0,8.82,2.667110,0.10855,15.08
2013-12-31,6000,17.15,30,6304.0,8.82,7.656025,0.10855,15.08
2013-12-31,6100,28.80,30,6304.0,8.82,18.756159,0.10855,15.08
2013-12-31,6200,49.10,30,6304.0,8.82,39.837882,0.10855,15.08


In [6]:
def regression(df):
  sns.scatterplot(x=df['Close'],y=df['bs_call'],hue=df['Expiration'])

In [7]:
def split_data(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Adj Close','Expiration','Price','svol','Vix']],df['Close'],test_size=0.2,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [8]:
def split_data_bs(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df[['Strike Price','Adj Close','Expiration','Price','svol','Vix']],df['bs_call'],test_size=0.2,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [9]:
n_units = 400
layers = 5
n_batch = 50
n_epochs = 20

In [10]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_df.shape[1]-2))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(LeakyReLU())

model.add(Dense(1, activation='relu'))
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='mse', optimizer=opt)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               2800      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 400)               1

In [12]:
def fit(X_train,y_train):
  history = model.fit(X_train, y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.2,
                    callbacks=[TensorBoard()],
                    verbose=1)

In [13]:
def metrics(y_test,y_pred):
  x=np.array(y_test).reshape(-1,1)
  y=np.array(y_pred).reshape(-1,1)
  reg=LinearRegression().fit(x,y)
  mae= mean_absolute_error(x,y)
  mse=mean_squared_error(x,y)
  print("REG_SCORE - ",reg.score(x,y))
  print("Mean Absolute Error -",mae)
  print("Mean Squared Error -",mse)
  print("Explained_Variance_Score - ",explained_variance_score(x,y))
  

In [14]:
def prediction(df):
  X_train, X_test, y_train, y_test=split_data(df)
  X_train_bs, X_test_bs, y_train_bs, y_test_bs=split_data_bs(df)
  fit(X_train,y_train)
  y_pred = model.predict(X_test)
  print("********Metrics of MLP MODEL********")
  metrics(y_test,y_pred)
  print("")
  print("********Metrics of BlackScholes Model********")
  metrics(y_test,y_test_bs.values)
  print("")

In [15]:
prediction(call_df)

Epoch 1/20
696/696 [==============================] - 14s 4ms/step - loss: 37972.0203 - val_loss: 12419.2578
Epoch 2/20
696/696 [==============================] - 2s 3ms/step - loss: 11595.2779 - val_loss: 10988.1553
Epoch 3/20
696/696 [==============================] - 2s 3ms/step - loss: 11226.4987 - val_loss: 11387.9355
Epoch 4/20
696/696 [==============================] - 2s 3ms/step - loss: 10228.6379 - val_loss: 9556.7598
Epoch 5/20
696/696 [==============================] - 2s 3ms/step - loss: 9537.1475 - val_loss: 9068.9658
Epoch 6/20
696/696 [==============================] - 2s 3ms/step - loss: 9073.3114 - val_loss: 9674.9443
Epoch 7/20
696/696 [==============================] - 2s 3ms/step - loss: 8989.1956 - val_loss: 8210.2373
Epoch 8/20
696/696 [==============================] - 2s 3ms/step - loss: 7700.9757 - val_loss: 7757.6992
Epoch 9/20
696/696 [==============================] - 2s 3ms/step - loss: 8098.5914 - val_loss: 9170.0762
Epoch 10/20
696/696 [=================

In [16]:
prediction(put_df)

Epoch 1/20
681/681 [==============================] - 2s 3ms/step - loss: 12923.3613 - val_loss: 7372.2090
Epoch 2/20
681/681 [==============================] - 2s 3ms/step - loss: 7234.6895 - val_loss: 6948.2427
Epoch 3/20
681/681 [==============================] - 2s 3ms/step - loss: 6621.0249 - val_loss: 6855.0449
Epoch 4/20
681/681 [==============================] - 2s 3ms/step - loss: 6822.6982 - val_loss: 7041.6030
Epoch 5/20
681/681 [==============================] - 2s 3ms/step - loss: 6378.1743 - val_loss: 8840.8926
Epoch 6/20
681/681 [==============================] - 2s 3ms/step - loss: 6259.6128 - val_loss: 8835.6094
Epoch 7/20
681/681 [==============================] - 2s 3ms/step - loss: 6323.4966 - val_loss: 6641.2920
Epoch 8/20
681/681 [==============================] - 2s 3ms/step - loss: 6255.6367 - val_loss: 7835.9287
Epoch 9/20
681/681 [==============================] - 2s 3ms/step - loss: 6356.3960 - val_loss: 6638.0830
Epoch 10/20
681/681 [========================